In [1]:
from typing import List
from custom_rag_loader import DbSupportedEmbeddingModels, RagConfig, SupportedModels, load_llm_huggingface_rag_model, DbSupportedChunkSizes, DbSupportedChunkOverlap
from enum import Enum
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.documents import Document

class USE_DB(Enum):
    TRUE = True
    FALSE = False


template = """[INST]You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. Always return the source of an information and it is mandatory to answer in GERMAN:

Context: {context}

Question: {question}[/INST]"""

prompt = ChatPromptTemplate.from_template(template)

config = RagConfig(
        n_ctx=4096,
        model=SupportedModels.Mistral_Finetune_Huggingface_RAFT,
        db_embedding_model=DbSupportedEmbeddingModels.Paraphrase_multilingual_MiniLM_L12_v2,
        db_chunk_size=DbSupportedChunkSizes.Chunk_1024,
        db_chunk_overlap=DbSupportedChunkOverlap.Overlap_128,
        version="v3",
        distance="l2",
        use_streaming=True,
    )

llm, db = load_llm_huggingface_rag_model(config)

retriever = db.as_retriever(k=3)


def format_docs(docs):
    context =""
    for doc in docs:
        context += "Content: \n" + doc.page_content + "\n"
        context += "Source: \n" + str(doc.metadata['file_path']) + "\n\n\n"
    return context

def simple_rag_chain(question: str, answer:str, use_db = USE_DB.TRUE) -> tuple[List[Document], str]:
    if(use_db):
        docs = retriever.get_relevant_documents(question)
        formatted_docs = format_docs(docs)
        final_prompt = prompt.format(context=formatted_docs, question= question)
        return (docs, llm.invoke(final_prompt))
    else:
        final_prompt = prompt.format(context=answer, question= question)
        return ([], llm.invoke(final_prompt))
    
def call_llm_huggingface(model: str, rag_chain, prompt, options, context):
    answer = context['vars']['answer']

    output = None
    documents = []

    documents, output = rag_chain(prompt, answer, USE_DB.TRUE)


    perplexity = None

    result = {
        'output': {
            'model': model,
            'output': output,
            'answer': answer,
            'keywords': context['vars']['keywords']['list'],
            'perplexity': perplexity,
            'timings': None,
            'token_usage': None,
            'finish_reason': None,
            'context': json.dumps([ob.__dict__ for ob in documents])
        }
    }

    return result


def call_api(prompt, options, context):

    return call_llm_huggingface('mistral-rag-instruct.HF-OLD', simple_rag_chain, prompt, options, context)





/home/tpllmws23/environments/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MODEL ID:  /home/tpllmws23/llms/raft


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.96s/it]


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1024,
  "repetition_penalty": 1.2,
  "temperature": 0.1,
  "top_p": 0
}



In [2]:
test = call_api('Welche Dokumente kÃ¶nnen als Nachweis fÃ¼r deutsche Sprachkenntnisse akzeptiert werden?', {}, {
    'vars': {'answer': 'gg','keywords': {'list': []}
    }
})

In [5]:
from pprint import pprint

pprint(test["output"]["output"])

('Human: [INST]You are a smart helpful assistant for the HTWG Konstanz. Answer '
 'the following question based only on the provided context. Always return the '
 'source of an information and it is mandatory to answer in GERMAN:\n'
 '\n'
 'Context: Content: \n'
 '\n'
 '§ 4 Sprachkenntnisse\n'
 '(1) 1Neben den allgemeinen Zugangsvoraussetzungen (§ 59 LHG) sind für die in '
 '§ 1 Abs. 1\n'
 'S. 1 genannten Studiengänge deutsche Sprach kenntnisse nachzuweisen. 2Diese '
 'können\n'
 'durch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich '
 'abgeschlossenes\n'
 'grundständiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\n'
 'Sprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht '
 'werden:\n'
 '1. Feststellungsprüfung für ein Bachelorstudium durch Vorlage der '
 'Zugangsberechtigung\n'
 'des Studienkollegs an der Hochschule Konstanz,\n'
 '2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt '
 'mindestens die\n'
 'Stufe TDN 4 erreic

In [6]:
print(test["output"]["output"])

Human: [INST]You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. Always return the source of an information and it is mandatory to answer in GERMAN:

Context: Content: 

§ 4 Sprachkenntnisse
(1) 1Neben den allgemeinen Zugangsvoraussetzungen (§ 59 LHG) sind für die in § 1 Abs. 1
S. 1 genannten Studiengänge deutsche Sprach kenntnisse nachzuweisen. 2Diese können
durch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes
grundständiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der
Sprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:
1. Feststellungsprüfung für ein Bachelorstudium durch Vorlage der Zugangsberechtigung
des Studienkollegs an der Hochschule Konstanz,
2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die
Stufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachprüfung für den Hochschulzugang (DSH), sofern die DSH 